<a href="https://colab.research.google.com/github/TemirlanKalim/SeugTest/blob/master/VolumeSeug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy
import math
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
data = pd.read_csv('TrainingDayVolume.csv',sep=';',skiprows=[0],names=['consumption_day','volume'],index_col=[0], engine='python') #Training Dataset (Больше данных - лучше)
#data.head()
scaler = MinMaxScaler(feature_range=(0, 1)) #задать границы трансформации
from google.colab import files

In [0]:
dataset = data.values
dataset = dataset.astype('float32')

In [0]:
dataset = scaler.fit_transform(dataset) #трансформировать датасет в 0-1
# split into train and test sets
train_size = int(len(dataset)) # тренировачный массив заданной длины
#test_size = len(dataset) - train_size
train = dataset[0:train_size,:] #создаем массив для тренировки модели
#, test , dataset[train_size:len(dataset),:]
#print(len(train), len(test))
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1, output_val): # функция для разбивки тренировачного датасета на инпут и аутпут
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-(output_val+1): #
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		b = dataset[(i+look_back):(i+look_back+output_val), 0] #
		dataY.append(b)
	return numpy.array(dataX), numpy.array(dataY)
# reshape into X=t and Y=t+1
look_back = 1800 #длина инпута
output_val= 365
trainX, trainY = create_dataset(train, look_back,output_val) # создание тренировочного инпута и аутпута
#testX, testY = create_dataset(test, look_back,output_val)
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1])) #трансформация инпут массива в подходящий для модели
#testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
# create and fit the LSTM network
model = Sequential() #создание модели Stacked LSTM
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(1, look_back))) # 1 LSTM уровень модели
model.add(LSTM(50, activation='relu')) # 2 LSTM уровень
model.add(Dense(output_val)) #Выходной уровень модели
model.compile(loss='mean_squared_error', optimizer='adam') #оптимизирование модели используя mean_squared_error и adam optimizer
model.fit(trainX, trainY, epochs=20, batch_size=2, verbose=2) # Запуск модели
# make predictions
trainPredict = model.predict(trainX) # Прогноз для тестирования
#testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict) # Обратная трансформация из 0-1 в привычный для прогноза
trainY = scaler.inverse_transform(trainY) # братная трансформация из 0-1 в привычный для тренировочного аутпута
#testPredict = scaler.inverse_transform(testPredict)
#testY = scaler.inverse_transform(testY)
# calculate root mean squared error
#trainScore = math.sqrt(mean_squared_error(trainY[:,0], trainPredict[:,0])) #вывод точности модели
#print('Train Score: %.2f RMSE' % (trainScore))
#testScore = math.sqrt(mean_squared_error(testY[:,0], testPredict[:,0]))
#print('Test Score: %.2f RMSE' % (testScore))
#model.save('testmodel.h5') #сохранение модели в h5 формате


In [0]:
#model=load_model('Volumemodel.h5')

In [0]:
frame = pd.read_csv('output1fINAL.csv',sep=';',index_col=[0],usecols=[0,1,2,3,4,5], engine='python') #загрузка инпут датасета
#frame,usecols=['consumption_day','volume']usecols=[0,1,2,3,4,5,6],

In [0]:
for i in range(51): #разбивка на 51 ПУ
  frame1=frame.loc[1+i*2006:2006+2006*i-3]
  frame1=frame1.tail(1800)
  #print(frame1.tail(1))
  frame1=frame1['volume']
  frame1=numpy.reshape(frame1.values, (frame1.values.shape[0],1))
  newset = frame1
  newset = newset.astype('float32')
  #print(newset)
  newset=scaler.fit_transform(newset)
  newset=numpy.reshape(newset, ( 1,1, newset.shape[0]))
  #print(newset)
  #print(newset.shape)
  res=model.predict(newset)
  #print(res.shape)
  res=scaler.inverse_transform(res)
  res=numpy.reshape(res, (res.shape[1], 1))
  ress=pd.DataFrame(data=numpy.round(res),columns=['volume'])
  ress['consumption_day'] = pd.date_range(start='6/27/2019', periods=len(ress), freq='D')
  ress=ress.assign(**{'counterparty_siug_id': frame.loc[[1+i*2006],['counterparty_siug_id']].values[0][0], 'gas_control_point_siug_id': i+1, 'gas_counter_siug_id': i+1})
  cols=ress.columns.tolist()
  cols=[cols[2],cols[3],cols[4],cols[1],cols[0]]
  ress=ress[cols]
  name='PuNumVolume'+str(i+1)+'.csv'
  ress.to_csv(name)
  files.download(name) 